# Collect text and links from Wiki articles

Example of API queries:
#links to Wiki API all plane text and lead

https://de.wikipedia.org/w/api.php?action=query&prop=extracts&titles=Gleichstellungsbeauftragte&exintro=&explaintext=&redirects=&exsentences=10&formatversion=2

https://de.wikipedia.org/w/api.php?action=query&prop=extracts&titles=Gleichstellungsbeauftragte&exintro=&explaintext=&redirects=&formatversion=2

In [2]:
#from wikitools2 import wiki, api #new version of wikitools
from wikitools import wiki, api
import urllib, json
import re, urlparse
import pprint
import mwparserfromhell
from mwparserfromhell.nodes import Wikilink
import pandas as pd

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
def check_categories(page):
    categories_f=["Kategorie:Frau"]
    categories_m=["Kategorie:Mann"]
    #http://de.wikipedia.org/w/api.php?action=query&titles=Hebamme&prop=categories
    l="de" 
    site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
    params={'action':'query','titles':page,'prop':'categories','format':'json','continue':'','redirects':''}
    request = api.APIRequest(site, params)
    for result in request.queryGen():
        if result["query"]["pages"].values()[0].get("categories"):
            for i in result["query"]["pages"].values()[0]["categories"]:
                if i["title"] in categories_f:
                    return "Frau" 
                elif i["title"] in categories_m:
                    return "Mann"   
        else:
            return None   
    return None
def query_de_wiki(site,params):
    req = api.APIRequest(site, params)
    res=req.query()
    return res["query"]["pages"][0]["extract"]
    
def get_links_from_text(title):   
    language="de"
    site = wiki.Wiki("http://" + language + ".wikipedia.org/w/api.php")
    params = {'action':'parse', 'page':title, 'prop':'wikitext'}
    req = api.APIRequest(site, params) 
  
    alllinklist = []
    male_person=[]
    female_person=[]
    all_text=""
    lead_text=""
    
    #store all raw text
    params={'action':'query','titles':title,'prop':'extracts','explaintext':'','redirects':'','formatversion':'2'}
    all_text=query_de_wiki(site,params)
            
    #store lead raw text
    params['exintro']=''
    lead_text=query_de_wiki(site,params)
            
    if True:
        #try:
        #res= req.query(querycontinue=False) #False ?
        for res in req.queryGen():
            #wiki text with links and text as wiki markup language
            text = res['parse']['wikitext']['*']
            #a=mwparserfromhell.parse(text)
            #print a.strip_code()

            #parse all links
            filtered_code = mwparserfromhell.parse(text).filter_wikilinks()
            for node in filtered_code: 
                link = node.title
                if ":" not in link:
                    if link is not None:
                        strlink = link.encode("utf-8").decode("utf-8").replace(' ', '_')
                        # strlink=urllib.quote(strlink.encode("utf-8"))
                        if strlink not in alllinklist:
                            alllinklist.append(strlink)
                            #Kategorie:Frau, Kategorie:Mann
                            #check the category of the link
                            gender=check_categories(strlink)
                            if gender =="Mann":
                                male_person.append(strlink)
                            elif gender=="Frau":
                                female_person.append(strlink)
                                

        return alllinklist,male_person,female_person,all_text,lead_text  
            
Dump = {} 
data=pd.read_csv("de/wiki/all_wiki_pages.csv",sep='\t', encoding='utf-8',index_col=0)   
pages=data.WikiName.values  
#TODO: check whether file exist
with open('de/wiki/all_data_from pages.json', 'r') as in_f:
    Dump=json.load(in_f)
print "We already stored", len(Dump), "professions", "out of ",len(pages)
#pages=["Gleichstellungsbeauftragte","Lehrer","Arzt"]
for title in pages:
    if title not in Dump:
        alllink,m,f,all_text,lead_text = get_links_from_text( title)
        print len(alllink)
        Dump.update({title:{"all_link":alllink,"male_person":m,"female_person":f,"male_person_amount":len(m),
                                "female_person_amount":len(f),"text":all_text,"lead":lead_text}})             

We already stored 885 professions out of  885


In [ ]:
with open('de/wiki/all_data_from pages.json', 'w') as out:
    json.dump(Dump, out, indent=4, sort_keys=True)  

# Store redirection bias ,gender of article and labour market in one file

In [10]:
import json
import pandas as pd

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f,encoding="utf-8")
all_wiki_pages=pd.read_csv("de/wiki/all_wiki_pages.csv",sep='\t', encoding='utf-8',index_col=0)##885
about_pages=load_simple_json('de/wiki/all_data_from pages.json')
about_pages=pd.DataFrame.from_dict(about_pages,orient="index")#885

#==============add Bias
about_pages["Bias"]=about_pages.apply(lambda x:all_wiki_pages[all_wiki_pages.WikiName.isin([x.name])].Bias.values[0],axis =1)

#==============add profession gender
import copy
f_val=load_simple_json('de/wiki/f_page_validated.json')
f_amb=load_simple_json('de/wiki/f_page_ambigious.json')
n_pages=load_simple_json('de/wiki/n_page_validated.json')
n_pages2=all_wiki_pages[(all_wiki_pages.Bias=="neutral")&(all_wiki_pages.Name!=all_wiki_pages.WikiName)].WikiName.values#10 neutral names of professions
n_pages_all=copy.deepcopy(n_pages)
n_pages_all.extend(n_pages2)
f_val_all=copy.deepcopy(f_val)
f_val_all.extend(f_amb)

def identify_gender(f_val_all,n_pages_all,x):
    if x.name in f_val_all:
        return "female"
    elif x.name in n_pages_all:
        return "neutral"
    else:
        return "male"
about_pages["prof_gender"]=about_pages.apply(lambda x: identify_gender(f_val_all,n_pages_all,x),axis=1)
#==check data
#print len(n_pages2)+len(n_pages)#11+33
#print len(f_val_all)#18 (6bias+12)
#all_wiki_pages[all_wiki_pages.Bias.isin(["neutral"])]#f+n=73(6+67)
#about_pages[about_pages.index.isin(set(set(all_wiki_pages[all_wiki_pages.Bias.isin(["neutral"])].WikiName)-set(n_pages_all))-set(f_val_all))]
#print about_pages[(about_pages.Bias=="male")&(about_pages.prof_gender!="male")]
#print about_pages[(about_pages.Bias=="female")&(about_pages.prof_gender!="female")]

In [5]:
all_together.Frauen

,Berufsgruppe,Frauen,KldB,KldB5,Männer,Overall,name,f_name
0,"Obj.Pers.Brandschutz,Arbeitssicherh(ssT)",37.700505,5318,53182,62.299495,8915,Hilfspolizist,Politesse
1,"Musik-,Gesangs-, Dirigententätigk. (ssT)",35.508637,9418,94183,64.491363,521,Audiodesigner,Audiodesignerin
2,"Aufsicht,Führung-TheaterFilm,Fernsehprod",40.187288,9449,94494,59.812712,5019,Filmgeschäftsführer,Filmgeschäftsführerin
3,Führung - Allgemeinbildende Schulen,62.997543,8419,84194,37.002457,2035,Direktor,Direktrice
4,Berufe für Post- und Zustelldienste,47.894039,5132,51321,52.105961,205963,Bote,Botin
5,Angehörige gesetzgebender Körperschaften,18.156809,7121,71214,81.843191,2908,Bundeskanzler,Bundeskanzlerin
6,Berufe in der Gemeindearbeit,58.613996,8332,83323,41.386004,10303,Diakon,Diakonin
7,"Berufe Informations,Telekommunikationst.",11.438161,2631,26313,88.561839,191884,Medientechniker,Medientechnikerin
8,"Berufe Informations,Telekommunikationst.",11.438161,2631,26313,88.561839,191884,Projektassistent,Projektassistentin
9,"Angehörige geistl. Orden, Mutterhäuser",62.571104,8333,83332,37.428896,879,Mönch,Nonne


In [11]:

#==============add labour market data
all_together=pd.read_csv("de/all_labour.csv",sep='\t', encoding='utf-8',index_col=0) 
list_of_extra_names=list(all_wiki_pages[all_wiki_pages.WikiName!=all_wiki_pages.Name].Name.values)#48
list_of_extra_wikinames=list(all_wiki_pages[all_wiki_pages.WikiName!=all_wiki_pages.Name].WikiName.values)


#male
df_most_m=all_together[all_together.Frauen<50.0]
df_most_m70=all_together[all_together.Frauen<70.0]
#wikipedia page names for male dominated professions
m_=list(all_wiki_pages[all_wiki_pages.Name.isin(df_most_m.name)].WikiName.values) #547
#female_names of male dominated professions
m_m=list(all_wiki_pages[all_wiki_pages.Oppositgender.isin(df_most_m.name)].WikiName.values) #6
print len(m_),len(m_m)
m_.extend(m_m)#553
print len(m_)#553


#another names of male prfessions 
m_most_m_other=df_most_m[df_most_m.name.isin(list_of_extra_names)].name#32
print len(m_most_m_other)#32
m_other=list(all_wiki_pages[all_wiki_pages.Name.isin(m_most_m_other)].WikiName.values)
m_.extend(m_other)#
print len(m_)#585
male_most_people=about_pages[about_pages.index.isin(m_)]#553
print len(male_most_people)

#===============================================

#female
#get where more than 50% of women involved
df_most_f=all_together[all_together.Frauen>=50.0]#311

print len(df_most_f)
f_=list(all_wiki_pages[all_wiki_pages.Name.isin(df_most_f.name)].WikiName.values) #306
#female names of female dominated professions
f_f=list(all_wiki_pages[all_wiki_pages.Oppositgender.isin(df_most_f.name)].WikiName.values) #12
print len(f_),len(f_f)
f_.extend(f_f)#318
print len(f_)
#other names f female professions
f_most_f_other=df_most_f[df_most_f.name.isin(list_of_extra_names)].name#7
f_other=list(all_wiki_pages[all_wiki_pages.Name.isin(f_most_f_other)].WikiName.values)
f_.extend(f_other)#
print len(set(f_)) #318
#309+535=844 over 885
female_most_people=about_pages[about_pages.index.isin(f_)]#318
print "",len(female_most_people)


print "No statistical data for ",len(list(set(set(about_pages.index)-set(m_))-set(f_)))," profesions:"
print list(set(set(about_pages.index)-set(m_))-set(f_))#14
#553+318=871 (871+14)=885

547 6
553
32
585
553
311
306 12
318
318
 318
No statistical data for  14  profesions:
[u'Fachpraktiker', u'Trainer', u'Techniker', u'Kaufmann', u'Sachverst\xe4ndiger', u'Verfahrensmechaniker', u'Facharbeiter', u'Angestellter', u'Fachleiter', u'Produktionshelfer', u'Helfer', u'Fachkraft', u'Meister', u'Fachkaufmann']


In [13]:
male_most_people["labor_market_domin"]=male_most_people.apply(lambda x: "male",axis=1)#"male"
female_most_people["labor_market_domin"]=female_most_people.apply(lambda x: "female",axis=1)#"female"
#all data
df_all_labour=pd.concat([male_most_people,female_most_people])

def do_smth(x):
    if x.name in df_all_labour.index.values:
        return df_all_labour[df_all_labour.index==x.name].labor_market_domin.values[0]
    else:
        return None
def return_woman_ratio_in_labour(x):
    #TODO
    if x.name in df_all_labour.index.values:
        return smth
    else:
        return None    

def get_labour_market_value(x,woman_ratio=True):
    if x in all_together.name.values:
        x_name=x
    else:
        opposit_x=all_wiki_pages[all_wiki_pages.WikiName==x].Oppositgender.values[0]
        if opposit_x in all_together.name.values:
            x_name=opposit_x
        else:
            x_name=all_wiki_pages[all_wiki_pages.WikiName==x].Name.values[0]
    if x_name in all_together.name.values:
        if woman_ratio:
            y=all_together[all_together.name==x_name].Frauen.values[0]
        else:
            y=all_together[all_together.name==x_name].Overall.values[0]
    else:
        y=None
    return y

about_pages["labor_market_domin"]=about_pages.apply(lambda x: do_smth(x),axis=1)
about_pages["profession"]=about_pages.apply(lambda x: x.name,axis=1)
    
    
about_pages["labor_market_woman"]=about_pages.apply(lambda x: get_labour_market_value(x.name) ,axis=1)
about_pages["labor_market_all"]=about_pages.apply(lambda x: get_labour_market_value(x.name,woman_ratio=False),axis=1)
#!!!!Store it in file==================================!!!!!!!!!!!!!!=====================
about_pages.to_csv('de/all_data_bias_labour_gender.csv', sep='\t', encoding='utf-8')

about_pages

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,male_person_amount,all_link,lead,text,male_person,female_person,female_person_amount,Bias,prof_gender,labor_market_domin,profession,labor_market_woman,labor_market_all
Abfallbeauftragter,0,"[Unternehmer, Kreislaufwirtschaft, Abfallbesei...",Ein Abfallbeauftragter oder Betriebsbeauftragt...,Ein Abfallbeauftragter oder Betriebsbeauftragt...,[],[],0,male,male,male,Abfallbeauftragter,26.126126,555
Abt,4,"[Kirchliche_Heraldik, Galero, Chorkleidung, Ha...","Ein Abt (von spätlat. abbas, aus aram. abba „V...","Ein Abt (von spätlat. abbas, aus aram. abba „V...","[Karl_Martell, Leo_X., Franz_I._(Frankreich), ...",[],0,male,male,female,Abt,62.571104,879
Account-Manager,0,"[Vertrieb, Kunde, Unternehmen, Key-Account-Man...","Der Account-Manager (AM, deutsch: Kundenbetreu...","Der Account-Manager (AM, deutsch: Kundenbetreu...",[],[],0,male,male,male,Account-Manager,43.248493,95238
Altenpflegehelfer,0,"[Hessen, Nordrhein-Westfalen, Baden-Württember...",Altenpflegehilfe mit der Berufsbezeichnung Alt...,Altenpflegehilfe mit der Berufsbezeichnung Alt...,[],[],0,male,male,female,Altenpflegehelfer,85.089472,502504
Altenpfleger,0,"[Ambulante_Pflege, Häusliche_Krankenpflege, So...",Altenpfleger und Altenpflegerinnen pflegen und...,Altenpfleger und Altenpflegerinnen pflegen und...,[],[],0,male,male,female,Altenpfleger,85.089472,502504
Amtsanwalt,0,"[Siegelmarke, Beamter_(Deutschland), Gehobener...",Ein Amtsanwalt ist in Deutschland ein Beamter ...,Ein Amtsanwalt ist in Deutschland ein Beamter ...,[],[],0,male,male,female,Amtsanwalt,93.380486,16542
Angestellter,7,"[Arbeitnehmer, Arbeiter, Arbeitsrecht, Sozialr...","Ein Angestellter, auch Privatangestellter, ist...","Ein Angestellter, auch Privatangestellter, ist...","[Fritz_Croner, Michel_Crozier, Arnd_Zschiesche...",[Irmgard_Keun],1,male,male,None,Angestellter,NaN,NaN
Animateur,0,"[Unterhaltungskünstler, Tourismus, Tagung, Ver...","Animateur (weibliche Form: Animateurin, von fr...","Animateur (weibliche Form: Animateurin, von fr...",[],[],0,male,male,female,Animateur,67.550274,8752
Anlagenelektriker,0,"[Berufsausbildung, Österreich, Elektrotechnik_...",Anlagenelektriker war ein Lehrberuf in Österre...,Anlagenelektriker war ein Lehrberuf in Österre...,[],[],0,male,male,male,Anlagenelektriker,3.414777,30778
Anlagenmechaniker,0,"[Ausbildungsberuf, Berufsbildungsgesetz_(Deuts...",Der Anlagenmechaniker ist in Deutschland ein s...,Der Anlagenmechaniker ist in Deutschland ein s...,[],[],0,male,male,male,Anlagenmechaniker,3.294255,39493


In [4]:
print "No labour market data for these professions (they are too general):"
about_pages[about_pages.labor_market_domin.isnull()]

No labour market data for these professions (they are too general):


,male_person_amount,all_link,lead,text,male_person,female_person,female_person_amount,Bias,prof_gender,labor_market_domin,profession
Angestellter,7,"[Arbeitnehmer, Arbeiter, Arbeitsrecht, Sozialr...","Ein Angestellter, auch Privatangestellter, ist...","Ein Angestellter, auch Privatangestellter, ist...","[Fritz_Croner, Michel_Crozier, Arnd_Zschiesche...",[Irmgard_Keun],1,male,male,None,Angestellter
Facharbeiter,0,"[Fachkraft, Ausbildung, Berufsausbildungsverhä...",Facharbeiter ist eine qualifizierte Fachkraft ...,Facharbeiter ist eine qualifizierte Fachkraft ...,[],[],0,male,male,None,Facharbeiter
Fachkaufmann,0,"[Aufstiegsfortbildung, Industrie-_und_Handelsk...",Der Fachkaufmann ist eine betriebswirtschaftli...,Der Fachkaufmann ist eine betriebswirtschaftli...,[],[],0,male,male,None,Fachkaufmann
Fachkraft,0,"[Eisenbahnbau, Sicherung_von_Zugfahrten, Beruf...","Eine Fachkraft ist allgemein eine Person, die ...","Eine Fachkraft ist allgemein eine Person, die ...",[],[],0,neutral,neutral,None,Fachkraft
Fachleiter,0,"[Lehrer, Schule, Studienseminar, Fachkonferenz...",Fachleiter bilden angehende Lehrer im Rahmen d...,Fachleiter bilden angehende Lehrer im Rahmen d...,[],[],0,male,male,None,Fachleiter
Fachpraktiker,0,"[Auszubildende, Berufsausbildung, Behinderung_...",Fachpraktiker bzw. Werker werden in Deutschlan...,Fachpraktiker bzw. Werker werden in Deutschlan...,[],[],0,male,male,None,Fachpraktiker
Helfer,6,"[Hilfe, Technisches_Hilfswerk, Helfer_vor_Ort,...","Helfer steht für:\neine Person, die in irgende...","Helfer steht für:\neine Person, die in irgende...","[Armin_Helfer, Christian_Helfer, Erwin_Helfer,...","[Monika_Helfer, Tricia_Helfer]",2,male,male,None,Helfer
Kaufmann,0,"[Kaufmann_(Begriffsklärung), Kaufmann_(Familie...","Ein Kaufmann ist eine Person, die erwerbsmäßig...","Ein Kaufmann ist eine Person, die erwerbsmäßig...",[],[],0,male,male,None,Kaufmann
Meister,0,"[Meister_(Begriffsklärung), Latein, Magister, ...","Meister (v. lat.: „Magister“ für „Lehrer“, eng...","Meister (v. lat.: „Magister“ für „Lehrer“, eng...",[],[],0,male,male,None,Meister
Produktionshelfer,0,"[Hilfsarbeiter, Arbeiter, Arbeitnehmerüberlass...",Ein Produktionshelfer ist ein Hilfsarbeiter im...,Ein Produktionshelfer ist ein Hilfsarbeiter im...,[],[],0,male,male,None,Produktionshelfer
